<a href="https://colab.research.google.com/github/Viyaaa/Movie-Recommender-System/blob/main/Movie%20Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation

Purpose: to recommend movies to user based on rating.



In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 62 bytes


In [ ]:
! kaggle datasets download -d sankha1998/tmdb-top-10000-popular-movies-dataset

  0% 0.00/1.31M [00:00<?, ?B/s]
100% 1.31M/1.31M [00:00<00:00, 113MB/s]


In [ ]:
! unzip /content/tmdb-top-10000-popular-movies-dataset.zip

Archive:  /content/tmdb-top-10000-popular-movies-dataset.zip
  inflating: TMDb_updated.CSV        


In [ ]:
import pandas as pd

In [ ]:
movies_df = pd.read_csv('/content/tmdb-top-10000-popular-movies-dataset.zip')
movies_df.head()

,Unnamed: 0,title,overview,original_language,vote_count,vote_average
0,0,Ad Astra,"The near future, a time when both hope and har...",en,2853,5.9
1,1,Bloodshot,"After he and his wife are murdered, marine Ray...",en,1349,7.2
2,2,Bad Boys for Life,Marcus and Mike are forced to confront new thr...,en,2530,7.1
3,3,Ant-Man,Armed with the astonishing ability to shrink i...,en,13611,7.1
4,4,Percy Jackson: Sea of Monsters,"In their quest to confront the ultimate evil, ...",en,3542,5.9


In [ ]:
df = movies_df.copy()

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              10000 non-null  object 
 1   overview           9970 non-null   object 
 2   original_language  10000 non-null  object 
 3   vote_count         10000 non-null  int64  
 4   vote_average       10000 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 390.8+ KB


In [ ]:
df.describe()

,vote_count,vote_average
count,10000.000000,10000.000000
mean,1020.825100,6.306230
std,1992.305005,1.354259
min,0.000000,0.000000
25%,143.000000,5.800000
50%,332.000000,6.500000
75%,926.250000,7.100000
max,25148.000000,10.000000


In [ ]:
df[df.isnull().any(axis=1)]

,title,overview,original_language,vote_count,vote_average
98,Recep İvedik 6,NaN,tr,21,4.4
394,Arjona Circo Soledad en Vivo,NaN,es,0,0.0
644,Ustica: The Missing Paper,NaN,it,26,5.6
696,Autonomes,NaN,fr,0,0.0
769,Tout nous sourit,NaN,fr,0,0.0
998,Vivre sans eux,NaN,fr,0,0.0
1438,La noche mágica,NaN,es,0,0.0
1582,The Water Monster,NaN,zh,1,10.0
1631,The Lion,NaN,fr,18,4.6
1650,Mucize 2: Aşk,NaN,tr,1,8.0


In [ ]:
df['original_language'].value_counts().sort_values(ascending=False)

en    8326
fr     385
ja     269
it     152
es     145
de      96
ko      92
cn      72
zh      71
hi      70
ru      69
da      32
sv      30
pt      22
id      21
ta      18
nl      13
no      13
tr      12
pl      10
tl       9
th       9
ar       6
sr       6
te       5
fa       5
he       4
cs       4
ml       4
hu       3
fi       3
af       2
el       2
ro       2
bs       1
sq       1
mk       1
xx       1
is       1
la       1
pa       1
nb       1
et       1
sh       1
eu       1
vi       1
ms       1
ka       1
mr       1
lv       1
ab       1
uk       1
Name: original_language, dtype: int64

**Best 3 Movies**

In [ ]:
df.nlargest(n=3, columns='vote_count')

,title,overview,original_language,vote_count,vote_average
80,Inception,"Cobb, a skilled thief who commits corporate es...",en,25148,8.3
128,Deadpool,Deadpool tells the origin story of former Spec...,en,22225,7.6
86,The Avengers,When an unexpected enemy emerges and threatens...,en,21893,7.7


**Best 3 English Language Movies**

In [ ]:
df[df['original_language'] == 'en'].nlargest(n=3, columns='vote_count')

,title,overview,original_language,vote_count,vote_average
80,Inception,"Cobb, a skilled thief who commits corporate es...",en,25148,8.3
128,Deadpool,Deadpool tells the origin story of former Spec...,en,22225,7.6
86,The Avengers,When an unexpected enemy emerges and threatens...,en,21893,7.7


**Best 3 France Language Movies**

In [ ]:
df[df['original_language'] == 'fr'].nlargest(n=3, columns='vote_count')

,title,overview,original_language,vote_count,vote_average
933,The Intouchables,A true story of two men who should never have ...,fr,11290,8.2
305,Amélie,"At a tiny Parisian café, the adorable yet pain...",fr,7266,7.9
2022,Serial (Bad) Weddings,A catholic French couple sees their life upsid...,fr,2477,6.8


**Best 3 Japan Language Movies**

In [ ]:
df[df['original_language'] == 'ja'].nlargest(n=3, columns='vote_count')

,title,overview,original_language,vote_count,vote_average
152,Spirited Away,"A young girl, Chihiro, becomes trapped in a st...",ja,9009,8.5
205,Your Name.,High schoolers Mitsuha and Taki are complete s...,ja,5225,8.5
170,Howl's Moving Castle,"When Sophie, a shy young woman, is cursed with...",ja,4755,8.4


**Best 3 Italy Language Movies**

In [ ]:
df[df['original_language'] == 'it'].nlargest(n=3, columns='vote_count')

,title,overview,original_language,vote_count,vote_average
464,Life Is Beautiful,A touching story of an Italian book seller of ...,it,8477,8.5
418,"The Good, the Bad and the Ugly",While the Civil War rages between the Union an...,it,4812,8.4
3925,Perfect Strangers,"During a dinner, a group of friends decide to ...",it,2984,7.9
